In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, avg, count, countDistinct, min as _min, month, year, current_date`


customers_df = spark.read.format("delta").load("abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/silver/customers/")
products_df = spark.read.format("delta").load("abfss://retailstream360@storageaccountsudip1.dfs.core.windows.net/silver/products/")
orders_df = spark.read.table("retail_catalog.silver.orders")
transactions_df = spark.read.table("retail_catalog.silver.transactions")

In [0]:
from pyspark.sql.functions import round
transactions_df=transactions_df.withColumn("total_price", round(col("price") * col("quantity"), 2))
transactions_df.display()

In [0]:
transactions_df.join(orders_df, "order_id") \
    .groupBy("customer_id").agg(_sum("total_price").alias("customer_revenue")).display()

In [0]:
total_revenue_df = transactions_df.agg(_sum("total_price").alias("total_revenue"))

revenue_by_product_df = revenue_by_product_df = transactions_df.groupBy("product_id").agg(_sum("total_price").alias("revenue"))

revenue_by_category_df = transactions_df.join(products_df, "product_id").groupBy("category").agg(_sum("total_price").alias("revenue"))

revenue_by_day_df = transactions_df.groupBy("transaction_date").agg(_sum("total_price").alias("daily_revenue"))

revenue_by_customer_df = transactions_df.join(orders_df, "order_id") \
    .groupBy("customer_id").agg(_sum("total_price").alias("customer_revenue"))

revenue_per_order_df = transactions_df.withColumn("total_price", col("price") * col("quantity")) \
    .groupBy("order_id").agg(_sum("total_price").alias("order_revenue"))

aov_df = revenue_per_order_df.agg(avg("order_revenue").alias("average_order_value"))